# Case Study of Traffic Prediction (Demand Prediction)
## Import packages and functions

In [1]:
import pandas as pd  
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def compute_metric(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true[np.where(y_true > 5)[0]], y_pred[np.where(y_true > 5)[0]])
    return mae, rmse, mape
  
def get_dataloader(X, y, device, bs, shuffle):
    return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.FloatTensor(X).to(device), 
                    torch.FloatTensor(y).to(device)), batch_size=bs, shuffle=shuffle, drop_last=False)

def calculate_metric_torch(true, pred, mask_value=5):
    mae = torch.mean(torch.abs(true - pred))
    rmse = torch.sqrt(torch.mean((pred - true) ** 2))
    if mask_value != None:
        mask = torch.gt(true, mask_value)
        pred = torch.masked_select(pred, mask)
        true = torch.masked_select(true, mask)
    mape = torch.mean(torch.abs(torch.div((true - pred), true)))
    return mae, rmse, mape

def trainer(model, lr, epochs, train_loader, val_loader, test_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    MSE = nn.MSELoss()
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = list()
        for src, trg in train_loader:
            pred = model(src)
            loss = MSE(pred, trg)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            epoch_loss.append(loss.item())
        epoch_loss = np.mean(epoch_loss)
        
        model.eval()
        best_rmse = epoch_loss
        with torch.no_grad():
            preds, trues = list(), list()
            for src, trg in val_loader:
                pred = model(src)
                preds.append(pred)
                trues.append(trg)
            mae, rmse, mape = calculate_metric_torch(torch.cat(trues), torch.cat(preds))

            if rmse < best_rmse:
                best_rmse = rmse
                preds, trues = list(), list()
                for src, trg in test_loader:
                    pred = model(src)
                    preds.append(pred)
                    trues.append(trg)
                test_mae, test_rmse, test_mape = calculate_metric_torch(torch.cat(trues), torch.cat(preds))
        
        print('Epoch %d, training loss: %.3f, validation mae: %.3f, rmse: %.3f, mape: %.3f' % (epoch, epoch_loss, mae, rmse, mape))
    
    return test_mae, test_rmse, test_mape


## Load datasets

In [3]:
datasets = np.load('./processed_demand_datasetsMAN.npz')
X_train, X_val, X_test, y_train, y_val, y_test = datasets['trainX'], datasets['valX'], datasets['testX'], datasets['trainy'], datasets['valy'], datasets['testy']


num_nodes = X_train.shape[0]
edges = np.load('./edges_GAman.npy')

adj = np.zeros((num_nodes, num_nodes))

for i in range(num_nodes): 
    adj[i,i] = 1
    
for i in range(len(edges)):
    adj[edges[i,0], edges[i,1]] = 1
    
adj = torch.LongTensor(adj)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## RF

In [12]:
rf = RandomForestRegressor(random_state=1000)
rf.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = rf.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
errors_rf = pd.DataFrame({'MAE': mae, 'RMSE': rmse, 'MAPE': mape}, index=['RF'])

print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))

test mae: 2.164, rmse: 4.394, mape: 21.693


## GBDT

In [13]:
gbdt = GradientBoostingRegressor(random_state=1000)
gbdt.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = gbdt.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
errors_gbdt = pd.DataFrame({'MAE': mae, 'RMSE': rmse, 'MAPE': mape}, index=['GBDT'])

print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))

test mae: 2.224, rmse: 4.432, mape: 22.182


## MLP

In [14]:
mlp = MLPRegressor(random_state=200, max_iter=400)
mlp.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = mlp.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
errors_mlp = pd.DataFrame({'MAE': mae, 'RMSE': rmse, 'MAPE': mape}, index=['MLP'])

print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))

test mae: 2.273, rmse: 4.587, mape: 23.100


## GRU

In [16]:
class GRU(nn.Module):
    def __init__(self, hidden_dim1, num_layers, hidden_dim2):
        super().__init__()
        self.GRU = nn.GRU(1, hidden_dim1, num_layers, batch_first=True)
        self.Dense = nn.Sequential(nn.Linear(hidden_dim1, hidden_dim2), nn.ReLU(), nn.Linear(hidden_dim2, 1))
    
    def forward(self, x):
        o, h = self.GRU(x)
        x = self.Dense(h[-1])
        return x.flatten()

In [18]:
train_loader = get_dataloader(X_train.reshape(-1,42,1), y_train.reshape(-1), device, 64, True)
val_loader = get_dataloader(X_val.reshape(-1,42,1), y_val.reshape(-1), device, y_test.reshape(-1).shape[0], True)
test_loader = get_dataloader(X_test.reshape(-1,42,1), y_test.reshape(-1), device, y_test.reshape(-1).shape[0], False)

#####################################################################################################################
# After class, you can try different learning rates and training epochs yourself. 
# Here for quick results, we just set the training epoch to 3.
lr = 0.001
epochs = 3
# For the sake of simplicity, we also omitted the parameter tuning process here. 
# The correct way is to adjust on the validation set to get the best hyperparameters, and then evaluate the model based on the test set.
#####################################################################################################################

model = GRU(64, 2, 32).to(device)

mae, rmse, mape = trainer(model, lr, epochs, train_loader, val_loader, test_loader)
errors_gru = pd.DataFrame({'MAE': mae.item(), 'RMSE': rmse.item(), 'MAPE': mape.item()}, index=['GRU'])

print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae.item(), rmse.item(), mape.item()))

Epoch 0, training loss: 136.957, validation mae: 3.005, rmse: 6.478, mape: 0.303
Epoch 1, training loss: 26.717, validation mae: 2.838, rmse: 6.337, mape: 0.284
Epoch 2, training loss: 25.051, validation mae: 3.144, rmse: 6.422, mape: 0.315
test mae: 2.526, rmse: 4.655, mape: 0.252


## GRU-GAT

In [19]:
class GATLayer(nn.Module):
    def __init__(self, device, in_dim, out_dim, alpha=0.2, dropout=0.5):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.dropout = dropout
        self.weights = nn.Parameter(torch.FloatTensor(in_dim, out_dim))
        nn.init.xavier_normal_(self.weights.data, gain=1.414)
        self.a = nn.Parameter(torch.FloatTensor(2*out_dim, 1))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

    def forward(self, h, adj):
        """
        h: (bs, num_node, in_dim)
        adj: (num_node, num_node)
        return (bs, num_node, out_dim)
        """
        bs, num_node, in_dim = h.size()
        src, trg = torch.nonzero(adj.long(), as_tuple=True)
        
        Wh = torch.matmul(h, self.weights) # (bs, num_node, out_dim)
        edge_h = torch.cat([Wh[:,src,:], Wh[:,trg,:]], dim=-1) # (bs, num_edges, 2*out_dim)
        edge_e = F.leaky_relu(torch.matmul(edge_h, self.a), negative_slope=self.alpha).squeeze(-1) # (bs, num_edges)

        attention = -9e15*torch.ones(bs, num_node, num_node).to(self.device) #（bs, num_node, num_node)
        attention[:, src, trg] = edge_e
        attention = F.dropout(F.softmax(attention, dim=-1), self.dropout) #（bs, num_node, num_node)

        h_prime = torch.einsum('bij,bjo->bio', attention, Wh)

        return h_prime
    
    
class GRUGAT(nn.Module):
    def __init__(self, device, in_dim, out_dim, num_node, adj):
        super().__init__()
        self.adj = adj
        self.gru = nn.GRU(1, in_dim, 2, batch_first=True)
        self.gat = GATLayer(device, in_dim, out_dim)
        self.end_conv = nn.Conv1d(out_dim, 1, kernel_size=1)
        
    def forward(self, x):
        # x: (bs, num_node, time_step)
        X = []
        for i in range(x.size(1)):
            o, h = self.gru(x[:,i,:].unsqueeze(-1))
            X.append(h[-1])
        X = torch.stack(X)
        X = self.gat(X.permute(1,0,2), self.adj) # (bs, num_node, 16)
        X = self.end_conv(X.permute(0,2,1))
        return X.squeeze(1)
    

For a complex model, such as the hybrid model of GAT and GRU used here, the training time is longer due to the larger number of parameters involved. The hyperparameter tuning process of complex models is usually more complicated, and some training tricks need to be involved, such as learning rate decay and early stopping mechanism. \
Here, also for simplicity and quick display of results, we only set the training epoch to 3 and set the learning rate to a constant.

In [20]:
train_loader = get_dataloader(X_train.transpose(1,0,2), y_train.T, device, 36, True)
val_loader = get_dataloader(X_val.transpose(1,0,2), y_val.T, device, 72, False)
test_loader = get_dataloader(X_test.transpose(1,0,2), y_test.T, device, 72, False)

lr = 0.01
epochs = 3

model = GRUGAT(device, 10, 128, 198, adj).to(device)

mae, rmse, mape = trainer(model, lr, epochs, train_loader, val_loader, test_loader)
errors_grugat = pd.DataFrame({'MAE': mae.item(), 'RMSE': rmse.item(), 'MAPE': mape.item()}, index=['GRU-GAT'])
print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae.item(), rmse.item(), mape.item()))


Epoch 0, training loss: 688.228, validation mae: 14.236, rmse: 22.770, mape: 0.618
Epoch 1, training loss: 612.897, validation mae: 12.425, rmse: 19.892, mape: 0.761
Epoch 2, training loss: 479.127, validation mae: 10.646, rmse: 19.284, mape: 0.652
test mae: 10.500, rmse: 19.010, mape: 0.645


## Error comparison

In [23]:
errors = pd.concat([errors_rf, errors_gbdt, errors_mlp, errors_gru, errors_grugat])
errors


,MAE,RMSE,MAPE
RF,2.163976,4.393530,21.693186
GBDT,2.223783,4.431532,22.182326
MLP,2.273329,4.587292,23.100089
GRU,2.525941,4.655260,0.252132
GRU-GAT,10.500470,19.010136,0.645191


## Use Colab to reproduce the above code on the cloud

If you have not configured the Pytorch environment (GPU version) on your personal computer, you may not be able to run this notebook 
or it will run very slowly. \
In this case, you might consider using Colab, a free-to-use cloud server developed by the Google Research team. \
\
The use of Colab is very simple, very similar to the operation of jupyter notebook, you can find the tutorials in the following links:  \
https://www.youtube.com/watch?v=inN8seMm7UI     \
https://research.google.com/colaboratory/faq.html   \
https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c  
\
I have uploaded the data and code related to travel demand forecasting in my Google drive, \
you can directly click the link below to run the code directly on the cloud through the Colab file I shared. \
There is no need to configure any environment, but you need to have a Google account.   \
https://drive.google.com/drive/folders/1gHIcapQ7ILvmnf8vgO0mKjN3NozgspR2?usp=sharing

After opening the link above, first you should right-click on the shared folder, select "Add shortcut to Drive", and then click on "My Drive" to save the shared folder to your Google Drive. Next, you should open the Demo.ipynb file with Google Colaboratory to run the above code on the cloud.
